# Importing and initializing links : 

In [ ]:
IMPORT numpy AS np
IMPORT random
IMPORT math
IMPORT networkx AS nx
IMPORT matplotlib.pyplot AS plt
IMPORT os
IMPORT time

# Define subdirectory for storing figures
SET fig_subdirectory TO "Figures"
IF NOT os.path.exists(fig_subdirectory):
    CALL os.makedirs(fig_subdirectory)

# Define subdirectory for writing data
SET data_subdirectory TO "Data"
IF NOT os.path.exists(data_subdirectory):
    CALL os.makedirs(data_subdirectory)

# Set precision for floating points
CALL np.set_printoptions(precision=4)

# Run the necessary notebooks to gain access to their methods
RUN Generate_Network_Topology.ipynb
RUN Generating_CRs.ipynb
RUN Links.ipynb


# Define network topology : 

In [ ]:
# Define edges for NSFNET and six_nodes topologies
SET nsfnet TO [
    (1, 2, 1100), (2, 3, 1600), (1, 3, 600), (2, 8, 1500), (1, 4, 1000),
    (4, 5, 600), (5, 7, 800), (7, 8, 700), (10, 11, 900), (4, 9, 1500),
    (9, 12, 800), (8, 10, 700), (10, 12, 500), (9, 14, 800), (10, 14, 500),
    (12, 13, 300), (13, 14, 300), (3, 6, 1000), (6, 5, 1100), (6, 13, 2000),
    (6, 11, 1200)
]
SET six_nodes TO [
    (5, 0, 1200), (0, 1, 1200), (1, 2, 800), (2, 3, 900), (3, 4, 1000),
    (4, 5, 900), (0, 4, 1000), (1, 3, 1000)
]

# Increase node values by one for six_nodes
SET six_nodes TO [(u+1, v+1, w) FOR (u, v, w) IN six_nodes]

# Set topology and edges
SET edges TO six_nodes
SET topology TO 'six_nodes'
SET dir_edges TO edges + [(v, u, w) FOR (u, v, w) IN edges]

# Create the network topology (graph)
SET g TO CALL create_bi_topology(dir_edges)
SET numNodes TO LENGTH OF g


# Simulation Parameters

In [ ]:
# Set ASLC
SET aslc TO "AWSL"

# Modify simulation parameters
SET numCR TO 25  # Number of initial CRs
SET k TO 3  # Given parameter
SET num_iter TO 1  # Average over
SET j_range TO 34  # X steps
SET factor TO 25  # Step multiplication factor

# Initialize arrays to store data of j_range X for num_iter
SET srcr TO np.zeros([num_iter, j_range])
SET tur TO np.zeros([num_iter, j_range])
SET nsp TO np.zeros([num_iter, j_range]).astype(int)

# Open file to write data
SET filename TO f"QKRA-{aslc}({topology}).txt"
SET data_path TO os.path.join(data_subdirectory, filename)
SET file TO open(data_path, "a")
CALL file.write(f"\n\nQKRA-{aslc}: {topology=}, k = {k}, Averaged over(z) = {num_iter} : \n")

PRINT "Running QKRA-{aslc} for numCR =", numCR, "with j_range =", j_range, "(factor =", factor, "), k =", k, ", iterations =", num_iter


# Main Simulation loop

In [ ]:
FOR z IN RANGE(num_iter):
    SET X TO numCR
    SET j TO 0

    WHILE j < j_range:
        # Initialize the links structure with weights
        (links, indices, ordered_indices) = CALL Links.initialize_links(dir_edges)

        # Generate X connection requests (CRs)
        SET CRs TO CALL CR.generate_crs(X)

        # Create the priority queue for CRs
        SET PQ_all TO CALL CR.create_priority_queue('all')

        FOR i IN [1, 2, 3]:
            FOR cr IN PQ_all[i]:
                # Find the k-shortest paths for the current CR
                SET k_sp TO CALL cr.k_sp(g, k)

                FOR l IN RANGE(k):
                    SET candidate_path TO k_sp[l][0]

                    # Attempt to allocate resources
                    SET success TO CALL Links.FF(candidate_path, cr, aslc)

                    IF success:
                        BREAK
                ##
            ##
        ##
        PRINT "All CRs for X =", X, "dealt with:";

        # Calculate metrics
        SET srcr[z, j] TO CALL CR.SRCR(CRs)
        SET nsp[z, j] TO CALL CR.NSP(CRs)
        SET tur[z, j] TO CALL Links.TUR()

        SET j TO j + 1
        SET X TO X + factor
    ##
    PRINT "Iteration", z+1, "complete"
    PRINT "#########################################################################################################################################\n"
##

PRINT "Simulation Ended"
FOR z IN RANGE(num_iter):
    SET X TO numCR
    SET j TO 0

    WHILE j < j_range:
        # Initialize the links structure with weights
        (links, indices, ordered_indices) = CALL Links.initialize_links(dir_edges)

        # Generate X connection requests (CRs)
        SET CRs TO CALL CR.generate_crs(X)

        # Create the priority queue for CRs
        SET PQ_all TO CALL CR.create_priority_queue('all')

        FOR i IN [1, 2, 3]:
            FOR cr IN PQ_all[i]:
                # Find the k-shortest paths for the current CR
                SET k_sp TO CALL cr.k_sp(g, k)

                FOR l IN RANGE(k):
                    SET candidate_path TO k_sp[l][0]

                    # Attempt to allocate resources
                    SET success TO CALL Links.FF(candidate_path, cr, aslc)

                    IF success:
                        BREAK
                ##
            ##
        ##
        PRINT "All CRs for X =", X, "dealt with:";

        # Calculate metrics
        SET srcr[z, j] TO CALL CR.SRCR(CRs)
        SET nsp[z, j] TO CALL CR.NSP(CRs)
        SET tur[z, j] TO CALL Links.TUR()

        SET j TO j + 1
        SET X TO X + factor
    ##
    PRINT "Iteration", z+1, "complete"
    PRINT "#########################################################################################################################################\n"
##

PRINT "Simulation Ended"


# Final Calculations and Plotting : 

In [ ]:
SET end_time TO time.time()
SET elapsed_time TO f"{int((end_time - start_time)/3600)}h {int((end_time - start_time)/60 % 60)}m {(end_time - start_time)%60 : .2f}s"
PRINT "Elapsed Time:", elapsed_time

SET X_range TO np.array(range(numCR, (j_range + 1)*factor, factor))

# Initialize arrays to calculate averages
SET avg_srcr TO np.zeros(j_range)
SET avg_tur TO np.zeros(j_range)
SET avg_nsp TO np.zeros(j_range)

FOR j IN RANGE(j_range):
    FOR i IN RANGE(num_iter):
        SET avg_srcr[j] TO avg_srcr[j] + srcr[i, j]
        SET avg_tur[j] TO avg_tur[j] + tur[i, j]
        SET avg_nsp[j] TO avg_nsp[j] + nsp[i, j]

SET avg_srcr TO avg_srcr / num_iter
SET avg_tur TO avg_tur / num_iter
SET avg_nsp TO avg_nsp / num_iter

FOR j IN RANGE(j_range):
    SET avg_srcr[j] TO round(avg_srcr[j], 4)
    SET avg_tur[j] TO round(avg_tur[j], 4)
    SET avg_nsp[j] TO round(avg_nsp[j], 4)

CALL file.write(f"\nX_range = [{', '.join(map(str, X_range))}] \nAverage_SRCR_{aslc} = [{', '.join(map(str, avg_srcr))}] \nAverage_TUR_{aslc} = [{', '.join(map(str, avg_tur))}] \nAverage_NSP_{aslc} = [{', '.join(map(str, avg_nsp))}]\n")
CALL file.close()

PRINT "Average SRCR:", avg_srcr
PRINT "Average TUR:", avg_tur
PRINT "Average NSP:", avg_nsp
PRINT "X:", X_range

SET X_i TO 25
SET X_f TO X_range[-1]
SET j_X_i TO int(X_i/25 - 1)
SET j_X_f TO int(X_f/25 - 1)
PRINT "Plotting for X =", X_i, "(j =", j_X_i, ") to X =", X_f, "(j =", j_X_f, ")"

# Plotting
SET fig, (ax1, ax2) TO plt.subplots(1, 2, figsize=(12, 6))
CALL ax1.scatter(X_range[j_X_i:j_X_f+1], avg_srcr[j_X_i:j_X_f+1], label='SRCR - scatter')
CALL ax1.scatter(X_range[j_X_i:j_X_f+1], avg_tur[j_X_i:j_X_f+1], label='TUR - scatter')
CALL ax2.scatter(X_range[j_X_i:j_X_f+1], avg_nsp[j_X_i:j_X_f+1])

CALL ax1.plot(X_range[j_X_i:j_X_f+1], avg_srcr[j_X_i:j_X_f+1], label='SRCR - line')
CALL ax1.plot(X_range[j_X_i:j_X_f+1], avg_tur[j_X_i:j_X_f+1], label='TUR - line')
CALL ax2.plot(X_range[j_X_i:j_X_f+1], avg_nsp[j_X_i:j_X_f+1])

CALL ax1.set_xlabel('X = Number of CRs')
CALL ax1.set_ylabel('Ratio')
CALL ax1.set_title("SRCR and TUR")
CALL ax1.minorticks_on()
CALL ax1.legend(loc='best')

CALL ax2.set_xlabel('X = Number of CRs')
CALL ax2.set_ylabel('NSP')
CALL ax2.set_title("NSP")
CALL ax2.minorticks_on()
CALL ax2.legend(loc='best')

CALL fig.suptitle(f"Network Evaluation for QKRA-{aslc} : averaged over {num_iter} iterations, k = {k}", y=1.05)

CALL plt.tight_layout()
CALL plt.show()

# Saving the plot
SET filename TO f"{aslc} - z = {num_iter}, k = {k} for X - {X_range[j_X_i]} to {X_range[j_X_f]}.jpeg"
PRINT filename

# Combine subdirectory and filename to create full path
SET fig_path TO os.path.join(fig_subdirectory, filename)
CALL plt.savefig(fig_path, dpi=300, bbox_inches='tight')
